# Chapter 3: RAG Part II: Chatting with Your Data

## Query Routing

Although using a single vector store is useful, the required data may live in a variety of data sources, including relational databases or other vector stores.

For example, you may have two vector stores: one for LangChain Python documentation and another for LangChain JS documentation. Given a user’s question, we would like to route the query to the appropriate inferred data source to retrieve relevant docs. _Query routing_ is a strategy used to forward a user’s query to the relevant data source.

### Logical Routing

In logical routing, we give the LLM knowledge of the various data sources at our disposal and then let the LLM reason which data source to apply based on the user’s query.

In order to achieve this, we make use of function-calling models like DeepSeek to help classify each query into one of the available routes. A _function call_ involves defining a schema that the model can use to generate arguments of a function based
on the query. This enables us to generate structured outputs that can be used to run other functions. The following Python code defines the schema for our router based on three docs for different languages:

In [2]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv

load_dotenv()


# data model
class RouteQuery(BaseModel):
    """
    Route a user query to the most relevant data source.
    """

    data_source: Literal["python_docs", "js_docs"] = Field(..., description="Given a user question, choose which data source would be most relevant for answering the question.")

# llm with function call
llm = ChatDeepSeek(model="deepseek-chat", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# prompt
system = "You're an expert at routing a user question to the appropriate data source. Based on the programming language the question is referring to, route it to the relevant data source."
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

# define router
router = prompt | structured_llm


Now we envoke the LLM to extract the data source based on the predefined schema:

In [3]:
question = """Why doesn't the following code work:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

response = router.invoke({"question": question})

response.data_source

'python_docs'

Once we’ve extracted the relevant data source, we can pass the value into another function to execute additional logic as required:

In [4]:
from langchain_core.runnables import RunnableLambda

def choose_route(result: RouteQuery) -> str:

    if "python_docs" in result.data_source.lower():
        return "chain for python_docs"
    return "chain for js_docs"

full_chain = router | RunnableLambda(choose_route)

Notice how we don’t do an exact string comparison but instead first turn the generated output to lowercase, and then do a substring match. This makes our chain more resilient to the LLM going off script and producing output that doesn’t quite conform to the schema we asked for.

Logical routing is most suitable when you have a defined list of data sources from which relevant data can be retrieved and utilized by the LLM to generate an accurate output. These can range from vector stores to databases and even APIs.